# 强化学习

## 核心概念与公式

- 贝尔曼方程 (Bellman Equation)：定义了当前状态（或状态-动作对）的价值与后续状态价值之间的关系，是多数 RL 算法的基础。
  - 状态价值函数
    $$
    V^{\pi}(s) = \mathbb{E}_{\pi}[R_{t+1} + \gamma V^{\pi}(S_{t+1})|S_t = s]
    $$
  - 动作价值函数
    $$
    Q^{\pi}(s, a) = r(s, a) + \gamma \sum_{s'}p(s' | s, a)V^{\pi}(s')
    $$
- Q-learning：一种经典的 off-policy 算法，直接学习最优动作价值函数 $Q^*(s, a)$。
  - 更新规则：$Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha[r_t + \gamma \mathop{max}\limits_{a} Q(s_{t+1},a) - Q(s_t, a_t)]$
- 策略梯度：直接对策略 $\pi_\theta(a|s)$ 进行优化。目标是找到一组参数 $\theta$，是的期望回报最大化。
  - 梯度公式：$ \nabla_\theta J(\theta) = \mathbb{E}_{\pi_\theta}[Q^{\pi_\theta}(s, a)\nabla_\theta \text{log}\pi_\theta(a|s)] $
- PPO (Proximal Policy Optimization)：目前 RLHF 中最主流的策略梯度算法，通过一个截断 (clipping) 的目标函数来限制每次策略更新的幅度，保证训练的稳定性。

## PPO (Proximal Policy Optimization)

PPO 的核心是在最大化期望回报的同时，避免策略更新过大导致训练崩溃。

- 目标函数：
  
    $$
    L^{CLIP}(\theta) = \hat{\mathbb{E}}_t[\text{min}(r_t(\theta)\hat{A}_t, \text{clip}(r_t(\theta), 1 - \epsilon, 1 + \epsilon)\hat{A}_t)] \\
    r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)}
    $$

    $\hat{A}_t$ 是优势函数 (Advantage Function)，表示在状态 $s_t$ 下选择动作 $a_t$ 比平均水平好多少。

    `clip`函数将概率比限制在 $[1 - \epsilon, 1 + \epsilon]$ 区间内。

- 工作机制：

    - 如果优势 $\hat{A}_t > 0$（好动作），目标函数鼓励增大 $r_t(\theta)$，但上限是$1 + \epsilon$。
    - 如果优势 $\hat{A}_t < 0$（坏动作），目标函数鼓励减小 $r_t(\theta)$，但下限是$1 - \epsilon$。

In [ ]:
# Policy Loss (Actor Loss)
import torch
import torch.nn as nn
from typing import Optional

class PolicyLoss(nn.Module):
    '''
    PPO Policy Loss
    '''
    def __init__(self, clip_eps: float = 0.2):
        super().__init__()
        self.clip_eps = clip_eps

    def forward(self, log_probs: torch.Tensor, old_log_probs: torch.Tensor, advantages: torch.Tensor) ->torch.Tensor:
        # 1. 计算新旧策略的概率比
        ratio = (log_probs - old_log_probs).exp()

        # 2. 计算两个目标项
        # 未截断的目标
        surr1 = ratio * advantages
        # 截断后的目标
        surr2 = torch.clamp(ratio, 1 - self.clip_eps, 1 + self.clip_eps) * advantages

        # 3. 取两者中较小的一个，并且加上负号（取最小值是为了防止过大的更新）
        return -torch.min(surr1, surr2).mean()
    
# Value Loss (Critic Loss)
class ValueLoss(nn.Module):
    '''
    PPO Value Loss
    '''
    def __init__(self, clip_eps: float = None):
        super().__init__()
        self.clip_eps = clip_eps

    def forward(self, values: torch.Tensor, old_values: torch.Tensor, returns: torch.Tensor) -> torch.Tensor:
        '''
        Args:
            values: 当前 Critic 网络的价值预测 V(s_t)
            old_values: 生成经验时旧 Critic 网络的价值预测
            return: 实际回报
        '''
        # 计算原始的均方误差损失
        loss = (values - returns) ** 2

        # Optional：对 value 也进行 clip，防止其变化过大
        if self.clip_eps is not None:
            values_clipped = old_values + torch.clamp(values - old_values, -self.clip_eps, self.clip_eps)
            loss_clipped = (values_clipped - returns) ** 2
            loss = torch.max(loss, loss_clipped)

        return loss.mean()


In [3]:
# 完整 PPO 训练循环
class PPOTrainer:
    def __init__(self):
        self.policy_loss_fn = PolicyLoss(clip_eps=0.2)
        self.policy_loss_fn = ValueLoss(clip_eps=0.2)

    def compute_total_loss(self, batch):
        # Actor loss
        policy_loss = self.policy_loss_fn(
            batch['log_probs'], # 现在策略
            batch['old_log_probs'], # 原有策略
            batch['advantages'] # 优势值
        )

        # Critic loss
        value_loss = self.value_loss_fn(
            batch['values'], # 当前 critic 网络预测的 value
            batch['old_values'], # 旧 critic 网络预测的 value
            batch['returns'] # 实际回报
        )

        # total loss
        total_loss = policy_loss + 0.5 * value_loss

        return{
            'total_loss': total_loss,
            'policy_loss': policy_loss,
            'value_loss': value_loss
        }

## DPO (Direct Preference Optimization)

DPO 是一种绕过显示奖励建模，直接根据偏好数据来优化 LLM 的方法。比传统的 RLHF 流程更加简单稳定。

- 核心思想：
  - DPO 的目标函数源于一个数学推导，表明标准的 RLHF 优化目标等价表示为一个二元交叉熵损失。这个损失函数的目标是最大化模型对“更优”回答的偏好，同时最小化对“更差”回答的偏好。
  - 损失函数：
    $$
    \mathcal{L}_{\text{DPO}}(\pi_\theta, \pi_{\text{ref}}) = - \mathbb{E}_{(x, y_w, y_l)\sim D}[\text{log} \sigma(\beta \text{log} \frac{\pi_\theta(y_w | x)}{\pi_{\text{ref}}(y_w | x)} - \beta \text{log} \frac{\pi_\theta(y_l | x)}{\pi_{\text{ref}}(y_l | x)})]
    $$
    - $y_w$ 是偏好的回答，$y_l$ 是不被偏好的回答
    - $\pi_\theta$ 是正在训练的模型，$\pi_{\text{ref}}$ 是固定的参考模型（第一阶段 SFT 训练出的模型）
    - $\beta$ 是控制 KL 散度强度的超参数

In [4]:
# DPO Loss
import torch
import torch.nn as nn
import torch.nn.functional as F

class DPOLoss(nn.Module):
    def __init__(self, beta: float = 0.1, label_smoothing: float = 0.0):
        super().__init__()
        self.beta = beta
        self.label_smoothing = label_smoothing

    def forward(self,
                policy_chosen_logps,
                policy_rejected_logps,
                reference_chosen_logps,
                reference_rejected_logps):
        
        # 计算策略模型和参考模型在 chosen / rejected 序列上的对数概率比
        pi_logratios = policy_chosen_logps - policy_rejected_logps
        ref_logratios = reference_chosen_logps - reference_rejected_logps

        # DPO core
        logits = pi_logratios - ref_logratios

        # calculate loss
        losses = - F.logsigmoid(self.beta * logits) * (1 - self.label_smoothing) - F.logsigmoid(-self.beta * logits) * self.label_smoothing
        
        # 计算隐式奖励，用于监控
        chosen_rewards = self.beta * (policy_chosen_logps - reference_chosen_logps).detach()
        rejected_rewards = self.beta * (policy_rejected_logps - reference_rejected_logps).detach()

        return losses.mean(), chosen_rewards, rejected_rewards

## GRPO (Grouped Relative Policy Optimization)

### 核心原理
1. 优势估计采用分组奖励归一化 (Grouped Reward Normalization)
    在 RLHF 中，通常会为一个 prompt 生成 K 个不同的回答，然后用奖励模型（Reward Model）为这些回答打分。传统的优势估计会在整个 batch 上进行归一化，会导致一个 prompt 中的高分回答拉高了整个 batch 的奖励基线，从而可能不公平地惩罚了另一个 prompt 的不错回答。GRPO 在组内（同一个 prompt 的 K 个回答中）进行奖励归一化，计算出的优势信号更稳定、更具有局部对比性
2. KL 散度作为显式的惩罚项
    在标准的 RLHF-PPO 流程中，为了防止策略模型 $\pi_\theta$ 偏离 SFT 的参考模型 $\pi_{\text{ref}}$ 太远，通常会将 KL 散度作为惩罚项加入到奖励信号中，即 $R = R_{\text{RM}} - \beta \cdot \text{RL}(\pi_{\theta}\Vert\pi_{\text{ref}})$。而在 GRPO 中，KL 散度被直接添加到策略优化目标函数的后面，作为一个独立的惩罚项，这使得优化目标更加清晰。

    其目标函数为：
    $$
    \hat{\mathbb{E}}_{q \sim P(Q), \{o_i\}_{i = 1}^G \sim \pi_{\theta_{\text{old}}}(o|q)} \lbrack\frac{1}{G}\sum_{i = 1}^G \frac{1}{|o_i|}\sum_{t = 1}^{|o_i|} \lbrace \text{min}(\frac{\pi_\theta(o_{i, t}|q, o_{i, < t})}{\pi_{\text{old}}(o_{i, t}|q, o_{i, < t})}\hat{A}_{i ,t}, \text{clip}(...)\hat{A}_{i,t}) \rbrace - \beta D_{\text{KL}}[\pi_\theta \Vert \pi_{\text{ref}}] \rbrack
    $$

In [ ]:
# GRPO implementation

import torch

def _get_per_token_logps(self, model, input_ids, attention_mask, logits_to_keep):
    '''
    计算每个 token 的对数概率

    Args:
        model：语言模型
        input_ids: 完整的输入序列 [batch, seq_len]
        attention_mask: 注意力掩码 [batch, seq_len] 
        logit_to_keep: 需要保留的 logits 数量（对应 completion 部分的长度）

    Returns:    
        torch.Tensor: 每个 token 的对数概率 [batch, logits_to_keep]
    '''
    # 1. 获取模型输出的 logits
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits  # [batch, seq_len, vocab_size]

    # 2. 提取 completion 部分的 logits
    # 由于 logits 是预测下一个 token 的概率分布，因此需要偏移一个位置
    completion_logits = logits[:, -(logits_to_keep + 1):-1, :] # [batch, logits_to_keep, vocab_size]
    completion_labels = input_ids[:, -logits_to_keep:]  # [batch, logits_to_keep]

    # 3. 计算每个 token 的对数概率
    log_probs = F.log_softmax(completion_logits, dim=-1) # [batch, logits_to_keep, vocab_size]，相当于对每个位置的 vocab 维度做 softmax
    
    # 4. 选择对应 token 的对数概率
    per_token_logps = log_probs.gather(dim=-1, index=completion_labels.unsqueeze(-1)).squeeze(-1) # [batch, logits_to_keep]

    return per_token_logps

def compute_grpo_loss(self, model, inputs):
    '''
    Args:
        model: 正在训练的策略模型
        inputs: 包含 prompt, completion, advantages 等数据的字典

    Returns:
        torch.Tensor: 计算输出的策略损失
    '''

    # 1. 准备输入 ID 和 attention mask
    prompt_ids, prompt_mask = inputs['prompt_ids'], inputs['prompt_mask']
    completion_ids, completion_mask = inputs['completion_ids'], inputs['completion_mask']
    input_ids = torch.cat([prompt_ids, completion_ids], dim = 1)
    attention_mask = torch.cat([prompt_mask, completion_mask], dim = 1)

    # 2. 计算当前策略模型对 completion 部分的每个 token 的对数概率
    logits_to_keep = completion_ids.size(1)
    per_token_logps = _get_per_token_logps(model, input_ids, attention_mask, logits_to_keep)

    # 3. 获取优势函数和旧策略的对数概率
    advantages = inputs['advantages'] # [batch,]
    old_per_token_logps = inputs['old_per_token_logps']

    # 4. 计算新旧策略概率比
    log_ratios1 = torch.exp(per_token_logps - old_per_token_logps) # [batch, logits_to_keep]

    # 5. 计算截断后的概率比
    log_ratios2 = torch.clamp(log_ratios1, 1 - self.clip_eps, 1+ self.clip_eps)

    # 6. 计算两个目标项
    surr1 = log_ratios1 * advantages.unsqueeze(-1) # [batch, logits_to_keep]
    surr2 = log_ratios2 * advantages.unsqueeze(-1) # [batch, logits_to_keep]

    per_token_loss = - torch.min(surr1, surr2) # [batch, logits_to_keep]

    return per_token_loss.mean()
